In [1]:
import torch

from llama_index.prompts import PromptTemplate
import os

os.environ["http_proxy"] = "http://192.168.1.254:7890"
os.environ["https_proxy"] = "http://192.168.1.254:7890"

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

LLAMA2_7B_chat = "/home/cymei/llama_hf/llama-2-chat-7b-hf"
LLAMA2_13B_chat = "/mnt/sdc/Llama-2-13b-chat-hf"
LLAMA2_7B_hf = "/home/cymei/llama_hf/llama-2-7b-hf"
openchat_35_0106 = "/mnt/nas1/models/openchat-3.5-0106"
chatglm_3_6B = "/mnt/nas1/models/chatglm3-6B"

selected_model = openchat_35_0106 #LLAMA2_7B_chat

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

In [2]:
from llama_index.llms import HuggingFaceLLM
from llama_index.output_parsers import LangchainOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(
        name = "Peptide name",
        description = "Provide the name of peptides with anti-inflammatory effects.",
        ),
    ResponseSchema(
        name = "Peptide sequence",
        description = "Provide the corresponding peptide sequence, and if not, fill in 0.",
    ),
    ResponseSchema(
        name = "reason",
        description = "Provide the reason why this peptide has anti-inflammatory properties.",
    ),
    ResponseSchema(
        name = "supportive context",
        description = "Provide content that supports the anti-inflammatory properties of this peptide.",
    ),
]

lc_output_parser = StructuredOutputParser.from_response_schemas(
    response_schemas
)
output_parser = LangchainOutputParser(lc_output_parser)

llm = HuggingFaceLLM(
    context_window = 8192,
    max_new_tokens = 4096,
    generate_kwargs = {"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt = query_wrapper_prompt,
    tokenizer_name = selected_model,
    model_name = selected_model,
    device_map = "auto",
    # output_parser = output_parser,
    # change these settings below depending on your GPU
    # model_kwargs = {"trust_remote_code": True} # "torch_dtype": torch.float16, "load_in_8bit": True,
)

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from llama_index import (
    ServiceContext,
    set_global_service_context,
)
 #载入模型以及embedding功能
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(
    model_name="/mnt/nas1/models/BAAI/bge-m3", # /mnt/nas1/models/BAAI/bge-m3  /home/cymei/bge-small-en
    device='cuda:0'
)  

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)
set_global_service_context(service_context)

In [4]:
from llama_index.llms import ChatMessage
import json
import re
from IPython.display import Markdown, display
    

file_loc = '/mnt/nas1/patent_data/anti-inflammation_peptide/parsed_pubmed.json'

with open(file_loc, 'r') as f:
    js_file = json.load(f)
k = 0

for item in js_file:
    #设置system_prompt
    system_str=  """You are a biologist and article expert. 
        You will be provided with some contents from a article and will be asked to answer specific questions related to the article. 
        Please answer the question only using the provided contents and do not make up the answer with prior knowledge."""
# The answer are based solely on the information provided in the given context and do not rely on any prior knowledge or assumptions.
    question="""
        Given the provided context, please identify all anti-inflammatory peptides mentioned. Output your results as: "peptide name", "peptide sequence", "relevant content"
        """
    # Please List the names, sequences, and related content of all anti-inflammatory peptides (motif sequence is not included) in the following format: [peptideName:'', peptideSequence:'', relatedContent:''].
    new_abstract = re.sub('\n', ' ', item['abstract'])
    # Given the provided context, please answer the query:
    context_str="""Context is :
            {}
            ---------------------
            {}
            """.format(new_abstract, question)  
    print(context_str)
    messages_dict = [
            # {"role": "system", "content": system_str},
            {"role": "user", "content": context_str},
        ]
    messages = [ChatMessage(**msg) for msg in messages_dict]
    response = llm.chat(messages)
    display(Markdown(f"<b>{response}</b>"))
    print('---------------------------------------------------------------------------------------------')
    k += 1
    if k == 30:
        break

Context is :
            BACKGROUND Atrial natriuretic peptide (ANP) secreted from atrial myocytes is shown to possess anti-inflammatory, anti-oxidant and immunomodulatory effects. The aim of this study is to assess the effect of ANP on bacterial lipopolysaccharide (LPS)-induced endotoxemia-derived neuroinflammation and cognitive impairment.   METHODS LPS (5 mg/kg) was given intraperitoneally to mice. Recombinant human ANP (rhANP) (1.0 mg/kg) was injected intravenously 24 h before and/or 10 min after LPS injection. Subdiaphragmatic vagotomy (SDV) was performed 14 days before LPS injection or 28 days before fecal microbiota transplantation (FMT). ANA-12 (0.5 mg/kg) was administrated intraperitoneally 30 min prior to rhANP treatment.   RESULTS LPS (5.0 mg/kg) induced remarkable splenomegaly and an increase in the plasma cytokines at 24 h after LPS injection. There were positive correlations between spleen weight and plasma cytokines levels. LPS also led to increased protein levels of ion

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<b>assistant: G

Peptide name: Atrial natriuretic peptide (ANP)
Peptide sequence: Not provided
Relevant content: ANP secreted from atrial myocytes is shown to possess anti-inflammatory, anti-oxidant and immunomodulatory effects.</b>

---------------------------------------------------------------------------------------------
Context is :
            High-fat diet (HFD)-induced comorbid cognitive and behavioural impairments are thought to be the result of persistent low-grade neuroinflammation. Metformin, a first-line medication for the treatment of type-2 diabetes, seems to ameliorate these comorbidities, but the underlying mechanism(s) are not clear. Pituitary adenylate cyclase-activating peptide (PACAP) and vasoactive intestinal peptide (VIP) are neuroprotective peptides endowed with anti-inflammatory properties. Alterations to the PACAP/VIP system could be pivotal during the development of HFD-induced neuroinflammation. To unveil the pathogenic mechanisms underlying HFD-induced neuroinflammation and assess metformin's therapeutic activities, (1) we determined if HFD-induced proinflammatory activity was present in vulnerable brain regions associated with the development of comorbid behaviors, (2) investigated if

<b>assistant: G

Pituitary adenylate cyclase-activating peptide (PACAP)</b>

...

---------------------------------------------------------------------------------------------
Context is :
            Pulmonary arterial hypertension (PAH) is an uncommon and deadly cardiopulmonary disease. PAH stems essentially from pulmonary artery (PA) remodeling induced predominantly by over-proliferation of PA smooth muscle cells (PASMCs) and inflammation. However, effective treatments are still missing in the clinic because the available drugs consisting of vasodilators are aimed to attenuate PAH symptoms rather than inhibit the remodeling process. Here, we aimed to specifically co-deliver apoptotic executor gene p53 and anti-inflammatory baicalein to PASMCs to alleviate PAH. The targeted co-delivery system was prepared through a carrier-free approach, which was prepared by loading the conjugate, NLS (nuclear localization signal) peptide-p53 gene, onto the baicalein pure crystals, followed by coating with glucuronic acid (GA) for targeting the glucose transport-1 (GLUT-1). The c

<b>assistant: G

The provided context does not mention any anti-inflammatory peptides.</b>

---------------------------------------------------------------------------------------------
Context is :
            BACKGROUND Neuroinflammation is the leading cause of neurological sequelae in ischemic stroke. Recently, we reported that the anti-inflammatory mediator annexin A1 (ANXA1) favored microglial M2 polarization in brain injury. The purpose of this study was to investigate electroacupuncture (EA) treatment and its potentially ANXA1-mediated anti-inflammatory effects in the middle cerebral artery occlusion/reperfusion (MCAO/R) mouse model of stroke.   METHODS Treatment with EA consisted of dense-sparse frequencies (alternating 4 Hz sparse waves for 1.5 s and 16 Hz dense waves for 1.5 s) at CV24 and GV26. Intracerebroventricular (ICV) injection of Boc-2 (5 µM) or short hairpin RNA (sh)ANXA1 (2 µL) 3 days before EA was performed to block the effects of ANXA1.   RESULTS EA pretreatment enhanced expression of ANXA1 and its receptor, formyl peptide receptor (FPR), when compared t

<b>assistant: G

Peptide name: annexin A1 (ANXA1)
Peptide sequence: Not provided
Relevant content: ANXA1 favored microglial M2 polarization in brain injury, EA treatment enhanced expression of ANXA1 and its receptor, formyl peptide receptor (FPR), EA-induced effects were either partially or completely prevented by prior administration of FPR antagonist Boc-2 or shANXA1.</b>

---------------------------------------------------------------------------------------------


In [1]:
from llama_index.llms.llama_api import LlamaAPI
import os

os.environ["http_proxy"] = "http://192.168.1.254:7890"
os.environ["https_proxy"] = "http://192.168.1.254:7890"

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

llama_api_key = 'LL-SxQGxNT1jWMZyQJgOQPNuOoUlAz1UhSkTqlYEicFUaPRajxavvY7jBbvYmN53ncI'

llm = LlamaAPI(api_key = llama_api_key, model = 'llama-70b-chat', temperature = 0.0)

In [2]:
from llama_index import (
    ServiceContext,
    set_global_service_context,
)
 #载入模型以及embedding功能
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(
    model_name="/mnt/nas1/models/BAAI/bge-m3", # /mnt/nas1/models/BAAI/bge-m3  /home/cymei/bge-small-en
    device='cuda:0'
)  

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)
set_global_service_context(service_context)

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
from llama_index.llms import ChatMessage
import json
import re
from IPython.display import Markdown, display
    

file_loc = '/mnt/nas1/patent_data/anti-inflammation_peptide/parsed_pubmed.json'

with open(file_loc, 'r') as f:
    js_file = json.load(f)
k = 0

for item in js_file:
    #设置system_prompt
    system_str=  """You are a biologist and article expert. 
        You will be provided with some contents from a article and will be asked to answer specific questions related to the article. 
        Please answer the question only using the provided contents and do not make up the answer with prior knowledge."""

    question="""
        The answer are based solely on the information provided in the given context and do not rely on any prior knowledge or assumptions.
        Use YES or NO to answer whether the given content contains any anti-inflammatory peptide sequence. Don't have redundant content.
        """
    # """GPT4 Correct User: Hello<|end_of_turn|>GPT4 Correct Assistant: Hi<|end_of_turn|>GPT4 Correct User: How are you today?<|end_of_turn|>GPT4 Correct Assistant:"""

    # Please List the names, sequences, and related content of all anti-inflammatory peptides (motif sequence is not included) in the following format: [peptideName:'', peptideSequence:'', relatedContent:''].
    new_abstract = re.sub('\n', ' ', item['abstract'])

    context_str="""Context is :
            {}
            ---------------------
            Given the provided context, please answer the query:
            {}
            """.format(new_abstract, question)  
    print(context_str)
    messages_dict = [
             {"role": "system", "content": system_str},
            {"role": "user", "content": context_str},
        ]
    messages = [ChatMessage(**msg) for msg in messages_dict]
    response = llm.chat(messages)
    display(Markdown(f"<b>{response}</b>"))
    print('---------------------------------------------------------------------------------------------')
    k += 1
    if k == 30:
        break

Context is :
            BACKGROUND Atrial natriuretic peptide (ANP) secreted from atrial myocytes is shown to possess anti-inflammatory, anti-oxidant and immunomodulatory effects. The aim of this study is to assess the effect of ANP on bacterial lipopolysaccharide (LPS)-induced endotoxemia-derived neuroinflammation and cognitive impairment.   METHODS LPS (5 mg/kg) was given intraperitoneally to mice. Recombinant human ANP (rhANP) (1.0 mg/kg) was injected intravenously 24 h before and/or 10 min after LPS injection. Subdiaphragmatic vagotomy (SDV) was performed 14 days before LPS injection or 28 days before fecal microbiota transplantation (FMT). ANA-12 (0.5 mg/kg) was administrated intraperitoneally 30 min prior to rhANP treatment.   RESULTS LPS (5.0 mg/kg) induced remarkable splenomegaly and an increase in the plasma cytokines at 24 h after LPS injection. There were positive correlations between spleen weight and plasma cytokines levels. LPS also led to increased protein levels of ion

<b>assistant: YES. The content mentions Atrial natriuretic peptide (ANP) as an anti-inflammatory peptide.</b>

---------------------------------------------------------------------------------------------
Context is :
            High-fat diet (HFD)-induced comorbid cognitive and behavioural impairments are thought to be the result of persistent low-grade neuroinflammation. Metformin, a first-line medication for the treatment of type-2 diabetes, seems to ameliorate these comorbidities, but the underlying mechanism(s) are not clear. Pituitary adenylate cyclase-activating peptide (PACAP) and vasoactive intestinal peptide (VIP) are neuroprotective peptides endowed with anti-inflammatory properties. Alterations to the PACAP/VIP system could be pivotal during the development of HFD-induced neuroinflammation. To unveil the pathogenic mechanisms underlying HFD-induced neuroinflammation and assess metformin's therapeutic activities, (1) we determined if HFD-induced proinflammatory activity was present in vulnerable brain regions associated with the development of comorbid behaviors, (2) investigated if

<b>assistant: The provided content does contain anti-inflammatory peptide sequences.

YES.</b>

---------------------------------------------------------------------------------------------
Context is :
            For food quality and safety issues, the emergence of foodborne pathogenic bacteria has further accelerated the spread of antibiotic residues and drug resistance genes. To alleviate the harm caused by bacterial infections, it is necessary to seek novel antimicrobial agents as biopreservatives to prevent microbial spoilage. Nanoantimicrobials have been widely used in the direct treatment of bacterial infections. CNMs, formed by chitosan nanoparticles and peptides, are promising antibiotic alternatives for use as excellent new antibacterial drugs against pathogenic bacteria. Herein, the current study evaluated the function of CNMs in the protection of foodborne pathogen Escherichia coli (E. coli) O157 infection using an intestinal epithelial cell model. Antibacterial activity assays indicated that CNMs exerted excellent bactericidal activity against E. coli O157. Assessme

...

---------------------------------------------------------------------------------------------
Context is :
            Pulmonary arterial hypertension (PAH) is an uncommon and deadly cardiopulmonary disease. PAH stems essentially from pulmonary artery (PA) remodeling induced predominantly by over-proliferation of PA smooth muscle cells (PASMCs) and inflammation. However, effective treatments are still missing in the clinic because the available drugs consisting of vasodilators are aimed to attenuate PAH symptoms rather than inhibit the remodeling process. Here, we aimed to specifically co-deliver apoptotic executor gene p53 and anti-inflammatory baicalein to PASMCs to alleviate PAH. The targeted co-delivery system was prepared through a carrier-free approach, which was prepared by loading the conjugate, NLS (nuclear localization signal) peptide-p53 gene, onto the baicalein pure crystals, followed by coating with glucuronic acid (GA) for targeting the glucose transport-1 (GLUT-1). The c

<b>assistant: YES. The content mentions baicalein, an anti-inflammatory compound.</b>

---------------------------------------------------------------------------------------------
Context is :
            BACKGROUND Neuroinflammation is the leading cause of neurological sequelae in ischemic stroke. Recently, we reported that the anti-inflammatory mediator annexin A1 (ANXA1) favored microglial M2 polarization in brain injury. The purpose of this study was to investigate electroacupuncture (EA) treatment and its potentially ANXA1-mediated anti-inflammatory effects in the middle cerebral artery occlusion/reperfusion (MCAO/R) mouse model of stroke.   METHODS Treatment with EA consisted of dense-sparse frequencies (alternating 4 Hz sparse waves for 1.5 s and 16 Hz dense waves for 1.5 s) at CV24 and GV26. Intracerebroventricular (ICV) injection of Boc-2 (5 µM) or short hairpin RNA (sh)ANXA1 (2 µL) 3 days before EA was performed to block the effects of ANXA1.   RESULTS EA pretreatment enhanced expression of ANXA1 and its receptor, formyl peptide receptor (FPR), when compared t

<b>assistant: The provided content does contain an anti-inflammatory peptide sequence. The sequence is ANXA1. Therefore, the answer is YES.</b>

---------------------------------------------------------------------------------------------


In [61]:
response_ans = response.message.content
print(response_ans)
answer_a = []
answer_b = []
task_b = {'peptideName':'', 'peptideSequence':'', 'relatedContent':''}

def extract_between(text, a, b):
    pattern = f"(?<={a}).*?(?={b})"
    matches = re.search(pattern, text)
    return matches.group()

task_a = (extract_between(response_ans, "A:", "\n"))
task_a = 'no'
if 'yes' in task_a.lower():
    answer_a.append(1)
    task_b['peptideName'] = extract_between(response_ans, "peptideName:", ",")
    task_b['peptideSequence']  = extract_between(response_ans, "peptideSequence:", ",")
    task_b['relatedContent']  = extract_between(response_ans, "relatedContent:", ".]")

elif 'no' in task_a.lower():
    answer_a.append(0)

else:
    answer_a.append(0.5)

answer_b.append(task_b)


print(answer_a)
print(answer_b)

G

Task A: YES

Task B: [peptideName: Atrial natriuretic peptide (ANP), peptideSequence: Not provided, relatedContent: Secreted from atrial myocytes, possesses anti-inflammatory, anti-oxidant and immunomodulatory effects, mitigates LPS-induced hippocampal inflammation and cognitive dysfunction through subdiaphragmatic vagus nerve-mediated gut microbiota-brain axis.]
[0]
[{'peptideName': '', 'peptideSequence': '', 'relatedContent': ''}]


In [65]:
print(answer_b[0]['peptideName'])

In [69]:
import pandas as pd
df = pd.DataFrame({"isHavePeptide":answer_a, "peptideName":answer_b[0]['peptideName'], "peptideSequence":answer_b[0]['peptideSequence'], "relatedContent":answer_b[0]['relatedContent']})
print(df)
df.to_csv('try.csv',index=False, mode='a')

   isHavePeptide peptideName peptideSequence relatedContent
0              0                                           
